In [4]:
# @title Optimizar GLB < 3 MB — SIN DRACO / SIN KTX2 (A-Frame & 3D Builder OK)
TARGET_MB = 3  # ← meta por defecto <3MB

from google.colab import files
import os, pathlib, subprocess

TARGET = TARGET_MB * 1024 * 1024

def sh(cmd, check=True):
    print(f"\n$ {cmd}")
    r = subprocess.run(cmd, shell=True, executable="/bin/bash",
                       stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(r.stdout)
    if check and r.returncode != 0:
        raise RuntimeError(f"Falló: {cmd}\n{r.stdout}")
    return r

def fsize(p):
    return os.path.getsize(p) if os.path.exists(p) else None

def human(n):
    if n is None: return "—"
    for u in ["B","KB","MB","GB","TB"]:
        if n < 1024: return f"{n:.1f}{u}"
        n /= 1024

print("== Instalando Node 20 + gltf-transform (con sharp para resize) ==")
try:
    sh("curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -")
    sh("sudo apt-get -y install nodejs")
except Exception:
    sh('bash -lc \'export NVM_DIR="$HOME/.nvm"; '
       'mkdir -p "$NVM_DIR"; '
       'curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.7/install.sh | bash; '
       '. "$NVM_DIR/nvm.sh"; nvm install 20; nvm use 20; node -v; npm -v\'')

# Solo CLI + sharp. Nada de Draco ni KTX2.
sh("npm i -g @gltf-transform/cli", check=False)
sh("npm explore @gltf-transform/cli -g -- npm i --include=optional sharp", check=False)

print("\n== Sube tu .glb ==")
up = files.upload()
SRC = next((k for k in up.keys() if k.lower().endswith(".glb")), None)
assert SRC, "❌ No subiste ningún .glb"
BNAME = pathlib.Path(SRC).stem
print(f"✅ Archivo: {SRC} ({human(fsize(SRC))})")

def step(cmd, infile, outfile, label, stop_if_target=True):
    print(f"\n🔧 {label}")
    sh(f'gltf-transform {cmd} "{infile}" "{outfile}"')
    sz = fsize(outfile)
    print(f"   → {outfile}  {human(sz)}")
    if stop_if_target and sz is not None and sz <= TARGET:
        print(f"🎯 Meta < {TARGET_MB} MB alcanzada en: {label}")
        return outfile, sz, True
    return outfile, sz, False

def meets(sz):
    return (sz is not None) and (sz <= TARGET)

curr = SRC
curr_sz = fsize(curr)

# 0) Reordenar buffers (mejor empaquetado general)
curr, curr_sz, hit = step("reorder", curr, f"{BNAME}_reorder.glb", "Reorder", stop_if_target=True)
if hit: final = curr

# 1) Limpieza (mantiene atributos clave para shading y UVs)
_, _, _ = step("weld",  curr, f"{BNAME}_weld.glb",  "Weld",  stop_if_target=False);  curr = f"{BNAME}_weld.glb"
_, _, _ = step("dedup", curr, f"{BNAME}_dedup.glb", "Dedup", stop_if_target=False);  curr = f"{BNAME}_dedup.glb"
curr, curr_sz, hit = step(
    "prune --keep-attributes position,normal,tangent,texcoord_0,texcoord_1,color_0",
    curr, f"{BNAME}_prune.glb", "Prune (mantén attrs clave)"
)
if hit: final = curr

# (Opcional) Generar tangentes si tu material usa normal maps y no existen:
# curr, curr_sz, hit = step("tangents", curr, f"{BNAME}_tangents.glb", "Generar tangentes")
# if hit: final = curr

# 2) Reducción de texturas progresiva (PNG/JPEG, sin KTX2)
for SIZE in [4096, 2048, 1024, 512, 256]:
    out, sz, hit = step(
        f"resize --width {SIZE} --height {SIZE}",
        curr, f"{BNAME}_tx{SIZE}.glb", f"Resize texturas {SIZE}x{SIZE}"
    )
    curr, curr_sz = out, sz
    if hit:
        final = curr
        break

# 3) Cuantización conservadora (respeta suavizado y curvaturas)
if not meets(curr_sz or 10**18):
    curr, curr_sz, hit = step(
        "quantize --quantize-position 14 --quantize-normal 10 --quantize-texcoord 12 --quantize-color 8 --pattern ALL",
        curr, f"{BNAME}_quant.glb", "Quantize (conservador)"
    )
    if hit: final = curr

# 4) Simplificación progresiva SUAVE (evita artefactos de malla)
if not meets(curr_sz or 10**18):
    for r in [0.95, 0.90, 0.85, 0.80, 0.78, 0.75]:
        out, sz, hit = step(
            f"simplify --ratio {r}",
            curr, f"{BNAME}_simp{int(r*100)}.glb", f"Simplify {r}"
        )
        curr, curr_sz = out, sz
        if hit:
            final = curr
            break

# 5) Último intento: resize duro a 256 (PNG/JPG, sin KTX2)
if not meets(curr_sz or 10**18):
    out, sz, hit = step(
        "resize --width 256 --height 256",
        curr, f"{BNAME}_tx256_final.glb", "Resize final 256x256"
    )
    curr, curr_sz = out, sz

final = curr
print("\n📦 Tamaños:")
print(" - Original:", SRC,   "→", human(fsize(SRC)))
print(" - Final:   ", final, "→", human(fsize(final)))

if not meets(fsize(final)):
    print(f"\n⚠️ No se alcanzó < {TARGET_MB} MB sin comprometer más geometría o texturas.")
    print("   Sugerencias: permitir ratios < 0.75 con cautela, o bajar texturas a 128x128 puntualmente.")

print("\n⬇️ Descargando:", final)
files.download(final)



== Instalando Node 20 + gltf-transform (con sharp para resize) ==

$ curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
2025-09-10 16:08:00 - Installing pre-requisites
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://deb.nodesource.com/node_20.x nodistro InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jam

Saving cuellos4_1.glb to cuellos4_1.glb
✅ Archivo: cuellos4_1.glb (9.7MB)

🔧 Reorder

$ gltf-transform reorder "cuellos4_1.glb" "cuellos4_1_reorder.glb"
info: prune: Removed types... Accessor (3)
info: cuellos4_1.glb (10.2 MB) → cuellos4_1_reorder.glb (10.2 MB)

   → cuellos4_1_reorder.glb  9.7MB

🔧 Weld

$ gltf-transform weld "cuellos4_1_reorder.glb" "cuellos4_1_weld.glb"
info: cuellos4_1_reorder.glb (10.2 MB) → cuellos4_1_weld.glb (10.2 MB)

   → cuellos4_1_weld.glb  9.7MB

🔧 Dedup

$ gltf-transform dedup "cuellos4_1_weld.glb" "cuellos4_1_dedup.glb"
info: cuellos4_1_weld.glb (10.2 MB) → cuellos4_1_dedup.glb (10.2 MB)

   → cuellos4_1_dedup.glb  9.7MB

🔧 Prune (mantén attrs clave)

$ gltf-transform prune --keep-attributes position,normal,tangent,texcoord_0,texcoord_1,color_0 "cuellos4_1_dedup.glb" "cuellos4_1_prune.glb"
info: cuellos4_1_dedup.glb (10.2 MB) → cuellos4_1_prune.glb (10.2 MB)

   → cuellos4_1_prune.glb  9.7MB

🔧 Resize texturas 4096x4096

$ gltf-transform resize --width 4

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>